# Proposition d'algorithme pour le choix des PFR

## Principe 

Un principe itératif:
1. on regarde pour chacune son ou ses projets préférés, puis on détermine tous les groupes valides que l'on peut faire à partir de là.
2. si aucune solution n'est retournée, on baisse d'un point les notes maximales (on passe toutes les notes 50 à 49 par exemple) et on recommence au point 1.
3. si au moins une solution est retournée, c'est fini. Il reste à choisir la solution en bonne intelligence...

De cette manière on cherche à satisfaire chacun au mieux, si ce n'est pas possible on *baisse un peu la barre* pour tout le monde et on recommence. Je trouve que c'est mieux que de chercher à maximiser une fonction globale qui pourrait satisfaire beaucoup la majorité et laisser quelques très déçus.

Par ailleurs, cet algorithme propose l'ensemble des (meilleures) possibilités, il n'y a pas d'aléatoire. L'implémentation est (assez) simple, mais un peu longue à tourner.

## Algorithme et exemple

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

class DataError(Exception):
    pass

Données exemples, aléatoires.

In [2]:
np.random.seed(88)
nb_projects = 4
group_sizes = [3, 4]
nb_persons = 4 * nb_projects
notation_types = [[50, 49, 1], [21, 20, 20, 19], [50, 25, 25]]
def random_row():
    notation = notation_types[np.random.randint(len(notation_types))]
    row = np.zeros(nb_projects)
    for n, i in zip(notation, np.random.permutation(nb_projects)):
        row[i] = n
    return row
dat = pd.DataFrame(data=[random_row() for i in range(nb_persons)],
                   columns=['Projet_{}'.format(i) for i in range(nb_projects)],
                   index=['Person_{}'.format(i) for i in range(nb_persons)])

In [3]:
dat

,Projet_0,Projet_1,Projet_2,Projet_3
Person_0,0,1,50,49
Person_1,50,0,1,49
Person_2,1,0,49,50
Person_3,50,1,0,49
Person_4,50,49,1,0
Person_5,50,25,0,25
Person_6,21,20,20,19
Person_7,1,50,0,49
Person_8,49,50,0,1
Person_9,50,0,49,1


Fonctions utiles.

In [4]:
def argsmax(serie, dat):
    '''List of max arguments'''
    m = serie.max()
    ids = []
    for i, s in enumerate(serie):
        if s == m:
            ids.append(i)
    return ids

def pref_proj(dat):
    '''Dictionary where each person points towards a list his/her of preferred projects
    For optimization purposes, we sort the projects from the most demanded to the less demanded'''
    ids_max = dat.apply(lambda s: argsmax(s, dat), axis=1)
    ps_max = {p: list((dat.columns[ids])) for p, ids in zip(dat.index, ids_max)}
    
    #dictionary (project, popularity)
    proj_pop = {p: 0 for p in dat.columns}
    #get for each project, the number of time it is a preferred project
    for ps in ps_max.values():
        for p in ps:
            proj_pop[p] += 1
    # sort the projects from most popular to less popular
    sorted_projs = sorted(proj_pop.items(), key=lambda x: x[1], reverse=True)
    #sort the dictionary values according to this order:
    ps_max = {p: [proj for proj, _ in sorted_projs if proj in p_projs] 
              for p, p_projs in ps_max.items()}
    return ps_max

Fonction principale de l'algorithme, qui explore l'arbre des possibilités.

In [5]:
def make_groups(dat, group_sizes, verbose=False):
    '''Returns a list of acceptable projects
    Optimization : start with most demanded projects and persons with the fewest preferred projects
    Parameters:
    -----------
    dat : a DataFrame with project notes for each person
    group_sizes : a list of acceptable group sizes
    verbose : activate mode verbose (for understanding the algorithm or debugging)
    Returns:
    --------
    a list of dictionaries of acceptable configurations
    '''
    def v(s, depth):
        '''Print verbose if asked'''
        if verbose:
            print(' ' * depth, end='')
            print(s)
            
    # par personne, liste de projet ayant la meilleure note
    prefs = pref_proj(dat)
    # (convenience) liste des personnes et des projets
    projects = list(dat.columns)
    
    # class persons according to their number of preferred projects
    p_nproj = {p: len(projs) for p, projs in prefs.items()}
    persons = [p for p, _ in sorted(p_nproj.items(), key=lambda x: x[1])]
            
    def insert_person(person, proj, acc):
        if len(acc[proj]) < max(group_sizes):
            new_acc = deepcopy(acc)
            new_acc[proj].append(person)
            return new_acc
        else:
            return None

    def rec(persons, acc, depth=0):
        '''Fonction récursive (non terminale, mais ça devrait passer...)
        Arguments:
        ----------
        persons : liste des personnes restantes à traiter
        acc : dictionnaire (projet, liste des personnes dans le projet) 
              contenant l'état de la solution actuelle
        Résultat:
        ---------
        Liste des solutions valables '''
        # fin de récursion si la liste des personnes est vide
        if not persons:
            # on teste si la solution est valable (taille des groupes)
            if all(len(g) in group_sizes for g in acc.values()):
                return [acc]
            else:
                return []
        # person en cours:
        person = persons[0]
        v('Les projets préférés de {} sont {}'.format(person, prefs[person]), depth)
        sol = []
        # pour chaque projet préféré de chaque personne, on crée un nouvelle branche
        for proj in prefs[person]:
            newacc = insert_person(person, proj, acc)
            # on test s'il a été possible d'ajouter la personne, sinon on ne crée pas de nouvelle branche
            if newacc:
                v('OK il y a de la place dans {}, on ajoute {},'
                  'et on continue la branche !'.format(proj, person), depth)
                # on effectue une union de listes
                sol = sol + rec(persons[1:], newacc, depth=depth+1)
            else:
                v('OH NON, il n\'y a plus de place dans {} pour {},'
                  'en effet il y a déjà : {} ! On s\'arrête là :('.format(proj, person, acc[proj]), depth)
        return sol
    sols = rec(persons, {proj:[] for proj in projects})
    return sols

Fonction utile qui cappe les valeurs à une certaine valeur :

In [6]:
def haircut(dat, n):
    ''' Caps all numbers at n'''
    dat_cap = dat.applymap(lambda x: min(n, x))
    return dat_cap

L'algorithme complet :

In [7]:
def algo(dat, group_sizes, cap=50):
    '''Main algorithm (recursive)'''
    # If the data is correct, this should not happen
    # put it there to prevent infinite loop
    if cap == -1:
        raise DataError('Bad data, no solution is possible')
    print("run for cap={}".format(cap))
    sol = make_groups(haircut(dat, cap), group_sizes)    
    if not sol:
        return algo(dat, group_sizes, cap=cap-1)
    return sol   

Test avec les données aléatoires :

In [8]:
algo(dat, [0, 3, 4])

run for cap=50
run for cap=49


[{'Projet_0': ['Person_11', 'Person_6', 'Person_10', 'Person_5'],
  'Projet_1': ['Person_15', 'Person_7', 'Person_4', 'Person_8'],
  'Projet_2': ['Person_12', 'Person_9', 'Person_0', 'Person_14'],
  'Projet_3': ['Person_13', 'Person_1', 'Person_3', 'Person_2']},
 {'Projet_0': ['Person_11', 'Person_6', 'Person_10', 'Person_5'],
  'Projet_1': ['Person_15', 'Person_7', 'Person_4', 'Person_8'],
  'Projet_2': ['Person_12', 'Person_9', 'Person_2', 'Person_14'],
  'Projet_3': ['Person_13', 'Person_1', 'Person_3', 'Person_0']},
 {'Projet_0': ['Person_11', 'Person_6', 'Person_10', 'Person_5'],
  'Projet_1': ['Person_15', 'Person_7', 'Person_4', 'Person_8'],
  'Projet_2': ['Person_12', 'Person_9', 'Person_2', 'Person_0'],
  'Projet_3': ['Person_13', 'Person_1', 'Person_3', 'Person_14']},
 {'Projet_0': ['Person_11', 'Person_6', 'Person_10', 'Person_5'],
  'Projet_1': ['Person_15', 'Person_13', 'Person_4', 'Person_8'],
  'Projet_2': ['Person_12', 'Person_9', 'Person_0', 'Person_14'],
  'Projet_3':

---

### Avec nos données :

Import et nettoyage des données.

In [9]:
import requests
from io import StringIO
r = requests.get('https://docs.google.com/spreadsheets/d/1hUWvO8wyEJL-_SkhgpdrwdYiDL7XQdrbTkcp21py8Lw/export?format=csv&id=1hUWvO8wyEJL-_SkhgpdrwdYiDL7XQdrbTkcp21py8Lw&gid=0')

In [10]:
dat_bgd = pd.read_csv(StringIO(r.text), skiprows=1, index_col='Nom', encoding='utf8')
dat_bgd = dat_bgd.loc[:, 'Clustaar':'Plume Labs']
dat_bgd = dat_bgd.fillna(0)
dat_bgd = dat_bgd.loc[[i for i in dat_bgd.index if isinstance(i, str)], :]

In [11]:
dat_bgd

,Clustaar,Kernix Lab,Total,BNP,Amaury,STIF,SFR 1,SFR 2,Alstom,SACEM,FootBar,DCBrain,IPSEN,GrDF,Plume Labs
Nom,,,,,,,,,,,,,,,
Julien Cloud,0,0,0,0,0,0,0,0,1,0,0,49,0,0,50
William BENHAIM,50,4,0,0,0,0,0,0,0,0,0,0,4,0,42
Guillaume MOHR,1,0,0,0,0,0,0,0,0,0,0,50,0,0,49
Jade Lu Dac,0,28,0,0,0,0,0,0,0,27,0,15,15,0,15
Olivier Large,49,0,0,0,0,0,0,0,1,0,0,50,0,0,0
Kim Pellegrin,0,40,0,0,0,0,0,0,0,10,0,50,0,0,0
Malik OUSSALAH,0,25,0,0,0,0,0,0,0,0,0,0,50,0,25
Catherine Verdier,12,8,10,0,0,12,0,0,2,0,12,12,10,10,12
Cyril Gilbert,0,0,50,5,0,20,0,0,0,0,25,0,0,0,0


Exécution de l'algorithme (attention, ça peut être long).

**Attention ** : bien penser à accepter des groupes de taille 0 !

In [12]:
algo(dat_bgd, [0,3,4])

run for cap=50
run for cap=49
run for cap=48
run for cap=47
run for cap=46
run for cap=45
run for cap=44
run for cap=43
run for cap=42
run for cap=41
run for cap=40
run for cap=39
run for cap=38
run for cap=37
run for cap=36
run for cap=35
run for cap=34
run for cap=33
run for cap=32
run for cap=31
run for cap=30
run for cap=29
run for cap=28
run for cap=27
run for cap=26


KeyboardInterrupt: 